## Geodesic Distances

Using the [fdasrsf](https://fdasrsf-python.readthedocs.io/en/latest/) library, we compute the geodesic distances between pairs of open curves.

#### Imports / Functions

In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy
import tqdm
from fdasrsf.geodesic import geod_sphere

In [3]:
def rescale_shell(x,y,v=3):
    # Code to scale a curve so that it lies between +-v/2 (default: 1.5) in both axes.
    p = max(max(y)-min(y),max(x)-min(x))
    q = p/v
    x_ = x/q
    y_ = y/q
    mp = min(x_)+(max(x_)-min(x_))/2
    x_ = x_-mp
    mp = min(y_)+(max(y_)-min(y_))/2
    y_ = y_-mp
    return x_,y_

#### Process Data

In [4]:
pth = "C:\\Users\\arian\\Documents\\GitHub\\distances.csv"
table = pd.read_csv(pth,header=None)

In [5]:
all_points = []
n = len(table)
print("n: "+str(n))
for j in range(0,n):
    coords = list(table.T[j])
    x = []
    y = []
    for i in range(0,len(coords)):
        if i%2 == 0:
            x.append(coords[i])
        else:
            y.append(coords[i])
            
    x,y = rescale_shell(np.array(x)[:-1],np.array(y)[:-1])

    F = np.column_stack((x,y)).T 
    all_points.append(F)

n: 440


#### Pairwise Distances

In [7]:
distances_DP = np.zeros((n,n))
errors = []

for i in tqdm.tqdm(range(0,n)):
    x1 = all_points[i][0]
    y1 = all_points[i][1]
    beta1 = np.column_stack([x1,y1]).T

    for j in range(i+1,n):
        x2 = all_points[j][0]
        y2 = all_points[j][1]
        
        beta2 = np.column_stack([x2,y2]).T
        
        try:
            d,_,_, = geod_sphere(beta1, beta2)
        except:
            try:
                d,_,_, = geod_sphere(beta2, beta1)
            except:
                print("Error for contours "+str(i)+" and "+str(j))
                errors.append([i,j])
                d = 100000
                
        distances_DP[i,j] = d
        distances_DP[j,i] = d
        
    pd.DataFrame(distances_DP).to_csv('Open_dists.csv')


print("Computed distances between "+str(n)+" contours, with "+str(len(errors))+" errors.")

100%|████████████████████████████████████████████████████████████████████████████| 440/440 [00:00<00:00, 440031.89it/s]

Computed distances between 440 contours, with 0 errors.


#### Saving Distances

In [8]:
dfInd = pd.read_csv("C:\\Users\\arian\\Documents\\GitHub\\Index.csv")
names = list(dfInd['Name'])
df = pd.DataFrame(distances_DP)

df.columns = names
df.index = names
df.index.name = 'Name'
df.to_csv('Open_Distances.csv')